## Assignment 5



Yunyi Zhang<BR>
USC ID: 7518630167

### Objective:
In this assignment, you'll work with the Amazon-Google product dataset to perform entity resolution. Your task is to identify and match records across the Amazon and Google product datasets that refer to the same product.

In [1]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from itertools import product
from numpy.linalg import norm
from collections import defaultdict
from Levenshtein import distance as lev
from copy import copy
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, roc_auc_score
from scipy.stats import ttest_rel

### Dataset: 
You'll use these files for this assignment:
- Amazon.csv
- Google.csv
- Amazon_GoogleProducts_perfectMapping.csv


### Question 1: Data Preprocessing and Feature Engineering

**1.1 [10 points]** : Load the three files using Pandas. How many unique products are there in (i) Amazon, and (ii) Google? (Hint: you will have to use the ground-truth links in the perfectMapping file to properly answer this question)


In [2]:
amazon = pd.read_csv('./Amazon-GoogleProducts/Amazon-GoogleProducts/Amazon.csv', encoding='unicode_escape', names = ['id','name','description','manufacturer','price'], header=0)
google = pd.read_csv('./Amazon-GoogleProducts/Amazon-GoogleProducts/GoogleProducts.csv', encoding='unicode_escape')
perfect_match = pd.read_csv('./Amazon-GoogleProducts/Amazon-GoogleProducts/Amzon_GoogleProducts_perfectMapping.csv', encoding='unicode_escape')

In [3]:
amazon.fillna(value=" ", inplace=True)
google.fillna(value=" ",inplace=True)
perfect_match.fillna(value=" ",inplace=True)

In [4]:
amazon.head(10)

,id,name,description,manufacturer,price
0,b000jz4hqo,clickart 950 000 - premier image pack (dvd-rom),,broderbund,0.00
1,b0006zf55o,ca international - arcserve lap/desktop oem 30pk,oem arcserve backup v11.1 win 30u for laptops ...,computer associates,0.00
2,b00004tkvy,noah's ark activity center (jewel case ages 3-8),,victory multimedia,0.00
3,b000g80lqo,peachtree by sage premium accounting for nonpr...,peachtree premium accounting for nonprofits 20...,sage software,599.99
4,b0006se5bq,singing coach unlimited,singing coach unlimited - electronic learning ...,carry-a-tune technologies,99.99
5,b000ehpzv8,emc retrospect 7.5 disk to disk windows,emc retrospect 7.5 disk to diskcromwindows,dantz,0.00
6,b00021xhzw,adobe after effects professional 6.5 upgrade f...,upgrade only; installation of after effects st...,adobe,499.99
7,b000gzwjgc,acad upgrade dragon naturallyspeaking pro solu...,- marketing information: dragon naturallyspeak...,nuance academic,399.54
8,b0000dbykm,mia's math adventure: just in time,in mia's math adventure: just in time children...,kutoka,19.99
9,b00029bqa2,disney's 1st & 2nd grade bundle (pixar 1st gra...,disney's 1st & 2nd grade bundle will help your...,disney,14.99


In [5]:
amazon.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1363 entries, 0 to 1362
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            1363 non-null   object 
 1   name          1363 non-null   object 
 2   description   1363 non-null   object 
 3   manufacturer  1363 non-null   object 
 4   price         1363 non-null   float64
dtypes: float64(1), object(4)
memory usage: 53.4+ KB


In [6]:
google

,id,name,description,manufacturer,price
0,http://www.google.com/base/feeds/snippets/1112...,learning quickbooks 2007,learning quickbooks 2007,intuit,38.99
1,http://www.google.com/base/feeds/snippets/1153...,superstart! fun with reading & writing!,fun with reading & writing! is designed to hel...,,8.49
2,http://www.google.com/base/feeds/snippets/1134...,qb pos 6.0 basic software,qb pos 6.0 basic retail mngmt software. for re...,intuit,637.99
3,http://www.google.com/base/feeds/snippets/1204...,math missions: the amazing arcade adventure (g...,save spectacle city by disrupting randall unde...,,12.95
4,http://www.google.com/base/feeds/snippets/1224...,production prem cs3 mac upgrad,adobe cs3 production premium mac upgrade from ...,adobe software,805.99
...,...,...,...,...,...
3221,http://www.google.com/base/feeds/snippets/1487...,jumpstart(r) advanced 1st grade,prepare your child for the 1st grade and beyon...,,19.99
3222,http://www.google.com/base/feeds/snippets/1491...,ibm(r) viavoice(r) advanced edition 10,ibm viavoice advanced edition release 10 is a ...,,78.95
3223,http://www.google.com/base/feeds/snippets/1497...,xbox 360: gears of war,as marcus fenix you fight a war against the im...,,59.99
3224,http://www.google.com/base/feeds/snippets/1498...,documents to go premium 7.0,this pda software enables you to use your docu...,,49.99


In [7]:
google.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3226 entries, 0 to 3225
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            3226 non-null   object
 1   name          3226 non-null   object
 2   description   3226 non-null   object
 3   manufacturer  3226 non-null   object
 4   price         3226 non-null   object
dtypes: object(5)
memory usage: 126.1+ KB


In [8]:
perfect_match

,idAmazon,idGoogleBase
0,b000jz4hqo,http://www.google.com/base/feeds/snippets/1844...
1,b00004tkvy,http://www.google.com/base/feeds/snippets/1844...
2,b000g80lqo,http://www.google.com/base/feeds/snippets/1844...
3,b0006se5bq,http://www.google.com/base/feeds/snippets/1842...
4,b00021xhzw,http://www.google.com/base/feeds/snippets/1843...
...,...,...
1295,b00005bigp,http://www.google.com/base/feeds/snippets/1773...
1296,b000h1df7w,http://www.google.com/base/feeds/snippets/1773...
1297,b000p9cr66,http://www.google.com/base/feeds/snippets/1772...
1298,b000p9cr66,http://www.google.com/base/feeds/snippets/1321...


In [9]:
unique_a=len(pd.unique(amazon['id']))
print(f'There are {unique_a} unique values in Amazon.')

There are 1363 unique values in Amazon.


In [10]:
unique_g=len(pd.unique(google['id']))
print(f'There are {unique_g} unique values in Google.')

There are 3226 unique values in Google.


**1.2 [15 points]**: Now, perform text preprocessing on the 'name', 'description', and 'manufacturer’ columns from both the Amazon and Google products datasets. Make sure to do the following preprocessing tasks:
- Convert all characters to lowercase.
- Remove special characters and punctuation, e.g., comma, parentheses, etc.
- Eliminate stop words, e.g.,”the”, “an”, “a”, etc.
- Convert empty strings and 'null' entries to actual null values.


After cleaning the text data, add three new columns to both of the dataframes: 'name_cleaned', 'description_cleaned', and 'manufacturer_cleaned'. These columns store the processed text.


(Although the following feature engineering processes are performed on the “cleaned” versions of the columns, it is always a good habit to not alter the original column, as it could be useful for spot-checks for your subsequent steps!)

In [11]:
def preprocess_text(text):
    processed_text = text.str.lower()
    #This is for question 2
    tokened_text = text.str.lower()

    english_stopwords = set(stopwords.words('english'))

    for i in range(len(processed_text)):
        tokens = word_tokenize(processed_text[i])
        tokens = [word for word in tokens if (word.isalpha() or word.isnumeric()) and word.isascii() and word not in english_stopwords]
        processed_text[i] = ' '.join(tokens)
        tokened_text[i] = tokens

    return processed_text, tokened_text

In [12]:
for column in ['name', 'description', 'manufacturer']:
    google[column + '_cleaned'], google[column + '_tokened'] = preprocess_text(google[column])
    amazon[column + '_cleaned'], amazon[column + '_tokened'] = preprocess_text(amazon[column])   

In [13]:
def token_text(text):
    tokened_text = copy(text)
    for i in range(len(text)):
        tokens = word_tokenize(tokened_text[i])
        tokens = [word for word in tokens]
        tokened_text[i] = tokens

    return tokened_text

In [14]:
google['tokened_name_raw'] = token_text(google['name'])
amazon['tokened_name_raw'] = token_text(amazon['name'])   

**1.3 [10 points]**: Apply TF-IDF transformations to the 'name_cleaned', 'description_cleaned', and 'manufacturer_cleaned' fields of both datasets. Append the resulting TF-IDF vectors to the respective datasets as new columns: 'name_tfidf', 'description_tfidf', and ‘manufacturer_tfidf'.

In [15]:
def tfidf_transform(df, column):

    vectorizer = TfidfVectorizer()
    X = np.array(vectorizer.fit_transform(df[column]).todense())
    matrix = [X[i] for i in range(X.shape[0])]
    tfidf = pd.Series(matrix, name='tfidf')

    return tfidf

In [16]:
full_joined_df = pd.concat([google, amazon], ignore_index=True)

In [17]:
vectorizer = TfidfVectorizer()
X = np.array(vectorizer.fit_transform(full_joined_df.iloc[:2,5]).todense())

In [18]:
for column in ['name_cleaned', 'description_cleaned', 'manufacturer_cleaned']:
    full_joined_df[column + '_tfidf'] = tfidf_transform(full_joined_df, column)

In [19]:
google = full_joined_df.iloc[:3226,:]
amazon = full_joined_df.iloc[3226:,:]

### Question 2: Simple Blocking

In this section, you will implement and apply various blocking techniques to efficiently identify potential matches between the Amazon and Google product datasets. Blocking reduces the comparison space by only comparing records that share certain characteristics, making the entity resolution process more scalable. Implement the following blocking strategies:


* Manufacturer Blocking: Create blocks of records that share the same manufacturer (i.e., where the records share the **exact string** value in the **manufacturer_cleaned** field). This blocking ensures that only records within each block, indicating they're from the same manufacturer, are to be compared with each other.


* Name Token Blocking: First, **tokenize** the 'name_cleaned' fields of the records in both datasets. Records are considered to be in the same block if they share **any common token** in their 'name' field. Consider these common tokens as the match key values that identify records belonging to the same block.


* Description Token Blocking: **Tokenize** the 'description_cleaned' fields of the records. Consider records as part of the same block if they share **any token** in their 'description_cleaned'. 


**2.1 [20 points]** As constraints,  1) discard any blocks that contain more than 50 records, and 2) delete any blocks that have no record from either one of the two dataset,, as a match requires at least a pair of records between the dataset. For the questions below, consider the collection of all the blocks generated from the above all three strategies, and answer the following:

### *The tokenized codes were included in Question 1

In [20]:
full_joined_df = pd.concat([google, amazon], ignore_index=True)
print('Google: '+str(google.shape))
print('Amazon: '+str(amazon.shape))
print('Full: '+str(full_joined_df.shape))

Google: (3226, 15)
Amazon: (1363, 15)
Full: (4589, 15)


In [21]:
manufacturer_blocks = full_joined_df.groupby('manufacturer_cleaned').apply(lambda x: x.index.tolist()).tolist()
manufacturer_blocks_constrained = [block for block in manufacturer_blocks if 1 <= len(block) <= 50
                       and any(0 <= i <= 3225 for i in block) and any(3226 <= i <= 4589 for i in block)]

In [22]:
def create_blocks(tokenizeds, word_blocks):
    
    for i, the_list in enumerate(tokenizeds):
        for word in the_list:
            word_blocks[word].append(i)

    the_blocks = list(word_blocks.values())

    return the_blocks

In [23]:
tokenized_names_cleaned = full_joined_df['name_tokened']
word_blocks_name_cleaned = defaultdict(list)

name_blocks_cleaned = create_blocks(tokenized_names_cleaned, word_blocks_name_cleaned)

name_blocks_constrained_cleaned = [
    block for block in name_blocks_cleaned if (
        1 <= len(block) <= 50
        and any(0 <= i <= 3225 for i in block)
        and any(3226 <= i <= 4589 for i in block)
    )
]

In [24]:
tokenized_descriptions = full_joined_df['description_tokened']
word_blocks_description = defaultdict(list)

description_blocks = create_blocks(tokenized_descriptions, word_blocks_description)

description_blocks_constrained = [
    block for block in description_blocks if (
        1 <= len(block) <= 50
        and any(0 <= i <= 3225 for i in block)
        and any(3226 <= i <= 4589 for i in block)
    )
]

1. How many total blocks were created across all blocking strategies after discarding any blocks that do not satisfy the constraint?

In [25]:
all_blocks = manufacturer_blocks_constrained + name_blocks_constrained_cleaned + description_blocks_constrained
all_blocks_before_constrains = manufacturer_blocks + name_blocks_cleaned + description_blocks

total_block = len(all_blocks)
print(f'{total_block} blocks were created.')

6401 blocks were created.


2. How many unique Amazon products are included in all of the blocks after applying the constraints?

In [26]:
unique_amazon_products = set(index for block in all_blocks for index in block if 3226 <= index <= 4589)
print(f'{len(unique_amazon_products)} unique Amazon products are included.')

1360 unique Amazon products are included.


3. What is the total number of blocks that only contain Amazon records across all blocking strategies before applying the constraints?

In [27]:
only_amazon_products = [block for block in all_blocks_before_constrains if all(3226 <= i <= 4589 for i in block)]
print(f'{len(only_amazon_products)} blocks only contain Amazon records.')

6174 blocks only contain Amazon records.


4. What about  the total number of blocks that only contain Google records before applying the constraints?

In [28]:
only_google_products = [block for block in all_blocks_before_constrains if all(0 <= i <= 3225 for i in block)]
print(f'{len(only_google_products)} blocks only contain Google records.')

5443 blocks only contain Google records.


### Question 3: Pairing and Similarity Calculation

**3.1 [15 points]**: Now consider the blocks generated by ***Manufacturer Blocking***. Within each valid block, generate all possible pairings of records from the Amazon dataset with those from the Google dataset. This step creates a set of candidate pairs, each potentially representing the same product. Using the ground-truth file we imported earlier,


1. What is the pairs completeness for the candidate pairs resulting from Manufacturer Blocking?
2. What reduction ratio has been achieved through Manufacturer Blocking?
3. Assuming perfect similarity, what will be the precision and recall rates for the overall ER system implementing Manufacturer Blocking strategy？


* Note: This method is efficient as it limits comparisons to records already deemed potentially identical or similar through blocking.


In [29]:
manufacture_pairs = set()

for block in manufacturer_blocks_constrained:
    block_pairs = set(product([full_joined_df.iloc[i,0] for i in block if 3226 <= i <= 4589],[full_joined_df.iloc[i,0] for i in block if 0 <= i <= 3225]))
    manufacture_pairs.update(block_pairs)

In [30]:
len(manufacture_pairs)

1456

In [31]:
ground_truth_pairs = set(zip(perfect_match['idAmazon'], perfect_match['idGoogleBase']))

In [32]:
true_positive_pairs = manufacture_pairs.intersection(ground_truth_pairs)
pairs_completeness = len(true_positive_pairs) / len(ground_truth_pairs)
print(f"Pairs Completeness is: {pairs_completeness}")

Pairs Completeness is: 0.03923076923076923


In [33]:
len(true_positive_pairs)

51

In [34]:
reduction_ratio = 1 - (len(manufacture_pairs) / (3226 * 1364))
precision = len(true_positive_pairs) / len(manufacture_pairs)
recall = len(true_positive_pairs) / len(ground_truth_pairs)

print(f"Reduction Ratio: {reduction_ratio}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")

Reduction Ratio: 0.9996691107624451
Precision: 0.03502747252747253
Recall: 0.03923076923076923


**3.2 [20 points]**: Consolidate the candidate pairs into a **new** single DataFrame referred to as "candidate_pairs". This DataFrame is intended to contain columns from the Amazon and Google datasets that correspond to each group of candidate pairs. Print the first 10 rows of "candidate_pairs".

For each pair, your DataFrame should at least include the following columns:

* 'amazon_id': The unique identifier for the product in the Amazon dataset.
* 'google_id': The unique identifier for the product in the Google dataset.
* 'amazon_name_tfidf': The name of the product from the Amazon dataset.
* 'google_name_tfidf': The name of the product from the Google dataset.
* 'amazon_description_tfidf': The description of the product from the Amazon dataset.
* 'google_description_tfidf': The description of the product from the Google dataset.
* 'amazon_manufacturer_tfidf': The manufacturer of the product from the Amazon dataset.
* 'google_manufacturer_tfidf': The manufacturer of the product from the Google dataset.


In [35]:
candidate_pairs = []

for amazon_id, google_id in manufacture_pairs:
    amazon_line = pd.DataFrame(amazon[amazon['id']==amazon_id])
    google_line = pd.DataFrame(google[google['id']==google_id])

    pair_data = {
        'amazon_id': amazon_line.iloc[0,0],
        'google_id': google_line.iloc[0,0],
        'amazon_name_tfidf': amazon_line.iloc[0,12],
        'google_name_tfidf': google_line.iloc[0,12],
        'amazon_description_tfidf': amazon_line.iloc[0,13],
        'google_description_tfidf': google_line.iloc[0,13],
        'amazon_manufacturer_tfidf': amazon_line.iloc[0,14],
        'google_manufacturer_tfidf': google_line.iloc[0,14]
    }

    candidate_pairs.append(pair_data)

candidate_pairs_df = pd.DataFrame(candidate_pairs)
candidate_pairs_df.head(10)

,amazon_id,google_id,amazon_name_tfidf,google_name_tfidf,amazon_description_tfidf,google_description_tfidf,amazon_manufacturer_tfidf,google_manufacturer_tfidf
0,b000fvqo0o,http://www.google.com/base/feeds/snippets/6247...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,b000cpshfs,http://www.google.com/base/feeds/snippets/1299...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,b00004ochj,http://www.google.com/base/feeds/snippets/1494...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,b000v7vz1u,http://www.google.com/base/feeds/snippets/4749...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,b000i82j80,http://www.google.com/base/feeds/snippets/3306...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
5,b000bcgfe2,http://www.google.com/base/feeds/snippets/4036...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
6,b000ayxld4,http://www.google.com/base/feeds/snippets/1836...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
7,b0009jlux8,http://www.google.com/base/feeds/snippets/1795...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
8,b0006b6b1a,http://www.google.com/base/feeds/snippets/1837...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
9,b000hbvvg4,http://www.google.com/base/feeds/snippets/1283...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [36]:
candidate_pairs_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1456 entries, 0 to 1455
Data columns (total 8 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   amazon_id                  1456 non-null   object
 1   google_id                  1456 non-null   object
 2   amazon_name_tfidf          1456 non-null   object
 3   google_name_tfidf          1456 non-null   object
 4   amazon_description_tfidf   1456 non-null   object
 5   google_description_tfidf   1456 non-null   object
 6   amazon_manufacturer_tfidf  1456 non-null   object
 7   google_manufacturer_tfidf  1456 non-null   object
dtypes: object(8)
memory usage: 91.1+ KB


**3.3 [10 points]**:Unfortunately, similarity functions are not perfect in practice. For the candidate set you computed in 3.1, calculate similarity scores for each candidate pair formed by: 

(i)  computing  the Cosine Similarity measure between each of the corresponding 'name_tfidf', 'description_tfidf', and 'manufacturer_tfidf' fields in the two records (leading to three cosine sim. scores); 

(ii)  average the three scores to yield a single composite ‘similarity’ score.


Next, we must compute a ‘threshold’ in [0,1], such that for pairs with score >= the threshold, the pair is predicted to be a duplicate; otherwise, the pair is deemed to be a non-duplicate. Consider potential thresholds at increments of 0.1, starting from 0 and ending at 1. At what threshold(s) is the F-measure optimized, and what is the optimized F-measure?


In [37]:
avg_similarities = []
name_similarities = []
description_similarities = []
manufacturer_similarities = []

for i in range(len(candidate_pairs_df)):
    name_vector_amazon = candidate_pairs_df.iloc[i, 2]
    name_vector_google = candidate_pairs_df.iloc[i, 3]

    description_vector_amazon = candidate_pairs_df.iloc[i, 4]
    description_vector_google = candidate_pairs_df.iloc[i, 5]

    manufacturer_vector_amazon = candidate_pairs_df.iloc[i, 6]
    manufacturer_vector_google = candidate_pairs_df.iloc[i, 7]

    name_sim = np.dot(name_vector_amazon, name_vector_google) / (
            norm(name_vector_amazon) * norm(name_vector_google))
    
    description_sim = np.dot(description_vector_amazon, description_vector_google) / (
            norm(description_vector_amazon) * norm(description_vector_google))
    
    manufacturer_sim = np.dot(manufacturer_vector_amazon, manufacturer_vector_google) / (
            norm(manufacturer_vector_amazon) * norm(manufacturer_vector_google))

    name_sim = np.nan_to_num(name_sim, nan=0)
    name_similarities.append(name_sim)

    description_sim = np.nan_to_num(description_sim, nan=0)
    description_similarities.append(description_sim)

    manufacturer_sim = np.nan_to_num(manufacturer_sim, nan=0)
    manufacturer_similarities.append(manufacturer_sim)

    avg_similarity = np.mean([name_sim, description_sim, manufacturer_sim], axis=0)
    avg_similarities.append(avg_similarity)

candidate_pairs_df['avg_similarity'] = avg_similarities
candidate_pairs_df['name_similarity'] = name_similarities
candidate_pairs_df['description_similarity'] = description_similarities
candidate_pairs_df['manufacturer_similarity'] = manufacturer_similarities

candidate_pairs_df.head(10)

C:\Users\云忆\AppData\Local\Temp\ipykernel_9332\2548815811.py:19: RuntimeWarning: invalid value encountered in double_scalars
  description_sim = np.dot(description_vector_amazon, description_vector_google) / (


,amazon_id,google_id,amazon_name_tfidf,google_name_tfidf,amazon_description_tfidf,google_description_tfidf,amazon_manufacturer_tfidf,google_manufacturer_tfidf,avg_similarity,name_similarity,description_similarity,manufacturer_similarity
0,b000fvqo0o,http://www.google.com/base/feeds/snippets/6247...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.548717,0.472945,0.173206,1.0
1,b000cpshfs,http://www.google.com/base/feeds/snippets/1299...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.333333,0.000000,0.000000,1.0
2,b00004ochj,http://www.google.com/base/feeds/snippets/1494...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.490402,0.294114,0.177091,1.0
3,b000v7vz1u,http://www.google.com/base/feeds/snippets/4749...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.372567,0.117702,0.000000,1.0
4,b000i82j80,http://www.google.com/base/feeds/snippets/3306...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.504328,0.512984,0.000000,1.0
5,b000bcgfe2,http://www.google.com/base/feeds/snippets/4036...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.362374,0.000000,0.087123,1.0
6,b000ayxld4,http://www.google.com/base/feeds/snippets/1836...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.362778,0.064323,0.024011,1.0
7,b0009jlux8,http://www.google.com/base/feeds/snippets/1795...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.333333,0.000000,0.000000,1.0
8,b0006b6b1a,http://www.google.com/base/feeds/snippets/1837...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.624055,0.634602,0.237562,1.0
9,b000hbvvg4,http://www.google.com/base/feeds/snippets/1283...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

In [38]:
thresholds = np.arange(0, 1, 0.1)
pred_pairs = set()

for threshold in thresholds:
    filtered_pairs = candidate_pairs_df[candidate_pairs_df['avg_similarity'] >= threshold]
    pred_pairs = set(zip(filtered_pairs['amazon_id'], filtered_pairs['google_id']))
    
    true_positive_pairs = pred_pairs.intersection(ground_truth_pairs)
    precision = len(true_positive_pairs) / len(filtered_pairs)
    recall = len(true_positive_pairs) / len(ground_truth_pairs)
    if (precision+recall)==0:
        f1_score_alg=0
    else:
        f1_score_alg = 2 * (precision * recall) / (precision + recall)
        
    print(f'Threshold: {round(threshold,1)} ----Precision:{round(precision,4)}; Recall: {round(recall,4)}; F1: {round(f1_score_alg,4)}')

Threshold: 0.0 ----Precision:0.035; Recall: 0.0392; F1: 0.037
Threshold: 0.1 ----Precision:0.035; Recall: 0.0392; F1: 0.037
Threshold: 0.2 ----Precision:0.035; Recall: 0.0392; F1: 0.037
Threshold: 0.3 ----Precision:0.035; Recall: 0.0392; F1: 0.037
Threshold: 0.4 ----Precision:0.0648; Recall: 0.0385; F1: 0.0483
Threshold: 0.5 ----Precision:0.0935; Recall: 0.0354; F1: 0.0513
Threshold: 0.6 ----Precision:0.2529; Recall: 0.0331; F1: 0.0585
Threshold: 0.7 ----Precision:0.3462; Recall: 0.0138; F1: 0.0266
Threshold: 0.8 ----Precision:0.25; Recall: 0.0008; F1: 0.0015
Threshold: 0.9 ----Precision:0.0; Recall: 0.0; F1: 0


**Threshold is 0.6, best f1 is 0.0585.**

## Assignment 6

### Objective:
In this follow-up assignment, you will build upon the entity resolution work from HW5. This time, you will incorporate additional similarity metrics and use machine learning models.

### Datasets:
You will continue to use the datasets from HW5:
- Amazon.csv
- Google.csv
- Amazon_GoogleProducts_perfectMapping.csv



### Question 1: More blocking keys 
**[10 points]** First, let’s repeat Question 3.2 and 3.3 in HW5, but using the other two blocking keys in Question 2 of HW5. What are their (threshold-optimized) F-measures and how do they compare to the answer in 3.2? Are the optimized thresholds different for each of the methods?

**Please only consider blocking strategy 2 (Name Token Blocking). You do not have to consider Description Token Blocking in this question.**

In [39]:
name_pairs = set()

for block in name_blocks_constrained_cleaned:
    block_pairs = set(product([full_joined_df.iloc[i,0] for i in block if 3226 <= i <= 4589],[full_joined_df.iloc[i,0] for i in block if 0 <= i <= 3225]))
    name_pairs.update(block_pairs)

In [40]:
new_candidate_pairs = []

for amazon_id, google_id in name_pairs:
    amazon_line = pd.DataFrame(amazon[amazon['id']==amazon_id])
    google_line = pd.DataFrame(google[google['id']==google_id])

    pair_data = {
        'amazon_id': amazon_line.iloc[0,0],
        'google_id': google_line.iloc[0,0],
        'amazon_name_tfidf': amazon_line.iloc[0,12],
        'google_name_tfidf': google_line.iloc[0,12],
        'amazon_description_tfidf': amazon_line.iloc[0,13],
        'google_description_tfidf': google_line.iloc[0,13],
        'amazon_manufacturer_tfidf': amazon_line.iloc[0,14],
        'google_manufacturer_tfidf': google_line.iloc[0,14]
    }

    new_candidate_pairs.append(pair_data)

new_candidate_pairs_df = pd.DataFrame(new_candidate_pairs)
new_candidate_pairs_df.head(10)

,amazon_id,google_id,amazon_name_tfidf,google_name_tfidf,amazon_description_tfidf,google_description_tfidf,amazon_manufacturer_tfidf,google_manufacturer_tfidf
0,b00004vx3p,http://www.google.com/base/feeds/snippets/1548...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,b000e65hki,http://www.google.com/base/feeds/snippets/1608...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,b0001gu7di,http://www.google.com/base/feeds/snippets/2243...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,b0009yegcu,http://www.google.com/base/feeds/snippets/1202...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,b000he9psc,http://www.google.com/base/feeds/snippets/5046...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.07787552772534471, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
5,b000ozjtoo,http://www.google.com/base/feeds/snippets/7343...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
6,b000ecqxjq,http://www.google.com/base/feeds/snippets/1828...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
7,b00004swln,http://www.google.com/base/feeds/snippets/1290...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
8,b0006b6b1a,http://www.google.com/base/feeds/snippets/1837...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
9,b000ap41x2,http://www.google.com/base/feeds/snippets/1140...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [41]:
avg_similarities = []
name_similarities = []
description_similarities = []
manufacturer_similarities = []

for i in range(len(new_candidate_pairs_df)):
    name_vector_amazon = new_candidate_pairs_df.iloc[i, 2]
    name_vector_google = new_candidate_pairs_df.iloc[i, 3]

    description_vector_amazon = new_candidate_pairs_df.iloc[i, 4]
    description_vector_google = new_candidate_pairs_df.iloc[i, 5]

    manufacturer_vector_amazon = new_candidate_pairs_df.iloc[i, 6]
    manufacturer_vector_google = new_candidate_pairs_df.iloc[i, 7]

    name_sim = np.dot(name_vector_amazon, name_vector_google) / (
            norm(name_vector_amazon) * norm(name_vector_google))
    
    description_sim = np.dot(description_vector_amazon, description_vector_google) / (
            norm(description_vector_amazon) * norm(description_vector_google))
    
    manufacturer_sim = np.dot(manufacturer_vector_amazon, manufacturer_vector_google) / (
            norm(manufacturer_vector_amazon) * norm(manufacturer_vector_google))

    name_sim = np.nan_to_num(name_sim, nan=0)
    name_similarities.append(name_sim)

    description_sim = np.nan_to_num(description_sim, nan=0)
    description_similarities.append(description_sim)

    manufacturer_sim = np.nan_to_num(manufacturer_sim, nan=0)
    manufacturer_similarities.append(manufacturer_sim)

    avg_similarity = np.mean([name_sim, description_sim, manufacturer_sim], axis=0)
    avg_similarities.append(avg_similarity)

new_candidate_pairs_df['avg_similarity'] = avg_similarities
new_candidate_pairs_df['name_similarity'] = name_similarities
new_candidate_pairs_df['description_similarity'] = description_similarities
new_candidate_pairs_df['manufacturer_similarity'] = manufacturer_similarities

new_candidate_pairs_df.head(10)

C:\Users\云忆\AppData\Local\Temp\ipykernel_9332\1853300151.py:22: RuntimeWarning: invalid value encountered in double_scalars
  manufacturer_sim = np.dot(manufacturer_vector_amazon, manufacturer_vector_google) / (
C:\Users\云忆\AppData\Local\Temp\ipykernel_9332\1853300151.py:19: RuntimeWarning: invalid value encountered in double_scalars
  description_sim = np.dot(description_vector_amazon, description_vector_google) / (


,amazon_id,google_id,amazon_name_tfidf,google_name_tfidf,amazon_description_tfidf,google_description_tfidf,amazon_manufacturer_tfidf,google_manufacturer_tfidf,avg_similarity,name_similarity,description_similarity,manufacturer_similarity
0,b00004vx3p,http://www.google.com/base/feeds/snippets/1548...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.089959,0.245919,0.023959,0.0
1,b000e65hki,http://www.google.com/base/feeds/snippets/1608...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.167237,0.329629,0.172081,0.0
2,b0001gu7di,http://www.google.com/base/feeds/snippets/2243...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.083040,0.194663,0.054457,0.0
3,b0009yegcu,http://www.google.com/base/feeds/snippets/1202...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.508495,1.000000,0.525485,0.0
4,b000he9psc,http://www.google.com/base/feeds/snippets/5046...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.07787552772534471, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.048970,0.146909,0.000000,0.0
5,b000ozjtoo,http://www.google.com/base/feeds/snippets/7343...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.047906,0.138068,0.005650,0.0
6,b000ecqxjq,http://www.google.com/base/feeds/snippets/1828...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.137249,0.288220,0.123527,0.0
7,b00004swln,http://www.google.com/base/feeds/snippets/1290...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.091819,0.152497,0.122961,0.0
8,b0006b6b1a,http://www.google.com/base/feeds/snippets/1837...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.624055,0.634602,0.237562,1.0
9,b000ap41x2,http://www.google.com/base/feeds/snippets/1140...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

In [42]:
thresholds = np.arange(0, 1.1, 0.1)

for threshold in thresholds:
    filtered_pairs = new_candidate_pairs_df[new_candidate_pairs_df['avg_similarity'] >= threshold]
    pred_pairs = set(zip(filtered_pairs['amazon_id'], filtered_pairs['google_id']))
    
    true_positive_pairs = pred_pairs.intersection(ground_truth_pairs)
    if len(filtered_pairs)>0:
        precision = len(true_positive_pairs) / len(filtered_pairs)
    else:
        precision=0

    recall = len(true_positive_pairs) / len(ground_truth_pairs)
    if (precision+recall)==0:
        f1_score_result=0
    else:
        f1_score_result = 2 * (precision * recall) / (precision + recall)
        
    print(f'Threshold: {round(threshold,1)} ----Precision:{round(precision,4)}; Recall: {round(recall,4)}; F1: {round(f1_score_result,4)}')

Threshold: 0.0 ----Precision:0.0386; Recall: 0.9423; F1: 0.0742
Threshold: 0.1 ----Precision:0.0979; Recall: 0.94; F1: 0.1773
Threshold: 0.2 ----Precision:0.216; Recall: 0.8469; F1: 0.3442
Threshold: 0.3 ----Precision:0.3189; Recall: 0.5492; F1: 0.4035
Threshold: 0.4 ----Precision:0.3237; Recall: 0.2338; F1: 0.2715
Threshold: 0.5 ----Precision:0.2952; Recall: 0.0892; F1: 0.137
Threshold: 0.6 ----Precision:0.3533; Recall: 0.0454; F1: 0.0804
Threshold: 0.7 ----Precision:0.4074; Recall: 0.0169; F1: 0.0325
Threshold: 0.8 ----Precision:0.5556; Recall: 0.0038; F1: 0.0076
Threshold: 0.9 ----Precision:0.25; Recall: 0.0008; F1: 0.0015
Threshold: 1.0 ----Precision:0; Recall: 0.0; F1: 0


**The best threshold is 0.3, best F1 is 0.4. The result is better than HW5.**

For the remainder of the questions below, assume the candidate set generated by the ***Name Token Blocking***.

In [43]:
new_candidate_pairs_df = pd.merge(new_candidate_pairs_df, amazon, how='left', left_on='amazon_id', right_on='id')
new_candidate_pairs_df = new_candidate_pairs_df[['amazon_id', 'google_id', 'avg_similarity', 'name_similarity', 'description_similarity','manufacturer_similarity','name', 'description', 'manufacturer', 'price']]
new_candidate_pairs_df.columns = ['amazon_id', 'google_id', 'avg_similarity', 'name_similarity', 'description_similarity','manufacturer_similarity','amazon_name', 'amazon_description', 'amazon_manufacturer', 'amazon_price']

new_candidate_pairs_df = pd.merge(new_candidate_pairs_df, google, how='left', left_on='google_id', right_on='id')
new_candidate_pairs_df = new_candidate_pairs_df[['amazon_id', 'google_id', 'avg_similarity', 'name_similarity', 'description_similarity','manufacturer_similarity','amazon_name', 'amazon_description', 'amazon_manufacturer', 'amazon_price', 'name', 'description', 'manufacturer', 'price']]
new_candidate_pairs_df.columns = ['amazon_id', 'google_id', 'avg_similarity', 'name_similarity', 'description_similarity','manufacturer_similarity','amazon_name', 'amazon_description', 'amazon_manufacturer', 'amazon_price', 'google_name', 'google_description', 'google_manufacturer', 'google_price']

### Question 2: More Similarity Metrics

**2.1 [15 points]** We will now explore a different way of computing similarity between the two records in a candidate pair. Calculate the normalized Levenshtein similarity for each pair of fields in the candidate pair (hint: in class, we discussed a method for converting Levenshtein distance to normalized Levenshtein similarity, such that it lies in [0,1]). Create three more columns in the "candidate_pairs" DataFrame: 'name_lev', 'description_lev', 'manufacturer_lev‘. 

Print the first five rows of the new "candidate_pairs" DataFrame.

In [44]:
def calculate_levenshtein(row):
    name_distance = lev(row['amazon_name'], row['google_name'])
    description_distance = lev(row['amazon_description'], row['google_description'])
    manufacturer_distance = lev(row['amazon_manufacturer'], row['google_manufacturer'])

    name_similarity = (1 - name_distance / max(len(row['amazon_name']), len(row['google_name']))) if max(len(row['amazon_name']), len(row['google_name'])) > 0 else 0
    description_similarity = (1 - description_distance / max(len(row['amazon_description']), len(row['google_description']))) if max(len(row['amazon_description']), len(row['google_description'])) > 0 else 0
    manufacturer_similarity = (1 - manufacturer_distance / max(len(row['amazon_manufacturer']), len(row['google_manufacturer']))) if max(len(row['amazon_manufacturer']), len(row['google_manufacturer'])) > 0 else 0

    return pd.Series([name_distance, description_distance, manufacturer_distance, name_similarity, description_similarity, manufacturer_similarity])

In [45]:
new_candidate_pairs_df[['name_lev', 'description_lev', 'manufacturer_lev', 'name_lev_similarity', 'description_lev_similarity', 'manufacturer_lev_similarity']] = new_candidate_pairs_df.apply(calculate_levenshtein, axis=1)

In [46]:
new_candidate_pairs_df.head(10)

,amazon_id,google_id,avg_similarity,name_similarity,description_similarity,manufacturer_similarity,amazon_name,amazon_description,amazon_manufacturer,amazon_price,google_name,google_description,google_manufacturer,google_price,name_lev,description_lev,manufacturer_lev,name_lev_similarity,description_lev_similarity,manufacturer_lev_similarity
0,b00004vx3p,http://www.google.com/base/feeds/snippets/1548...,0.089959,0.245919,0.023959,0.0,easy drums (jewel case),easy drums is an interactive guide that will s...,arcmedia,9.99,roxio(r) easy media creator(r) 9 suite,the roxio easy media creator 9 suite lets you ...,,99.99,29.0,368.0,8.0,0.236842,0.244353,0.000000
1,b000e65hki,http://www.google.com/base/feeds/snippets/1608...,0.167237,0.329629,0.172081,0.0,punch! home design architectural series 4000 v10,punch! home design architectural series 4000 b...,punch! software,199.99,punch software 38100 - punch! super home suite...,punch software 38100 : the premium home design...,,45.97,113.0,580.0,14.0,0.236486,0.220430,0.066667
2,b0001gu7di,http://www.google.com/base/feeds/snippets/2243...,0.083040,0.194663,0.054457,0.0,mavis beacon teaches typing deluxe 16,mavis beacon teaches typing 16 deluxe offers t...,broderbund,39.99,individual software prm-wn5 professor teaches ...,professor teaches windows xp prm-wn5 individua...,individual software,24.99,43.0,422.0,18.0,0.232143,0.096360,0.052632
3,b0009yegcu,http://www.google.com/base/feeds/snippets/1202...,0.508495,1.000000,0.525485,0.0,instant architect design suite,instant architect design suite (win 98 me nt 2...,imsi design,29.99,instant architect design suite (00ids510cc),instant architect design suite brand: imsi des...,,28.79,13.0,68.0,10.0,0.697674,0.381818,0.090909
4,b000he9psc,http://www.google.com/base/feeds/snippets/5046...,0.048970,0.146909,0.000000,0.0,the print shop 22 pro publisher deluxe sb cs b...,the print shop 22 pro publisher deluxe deliver...,encore software,99.95,encore software 11231 - hoyle bridge club sb c...,encore software 11231 : encore software hoyle ...,,25.97,45.0,1168.0,14.0,0.328358,0.061093,0.066667
5,b000ozjtoo,http://www.google.com/base/feeds/snippets/7343...,0.047906,0.138068,0.005650,0.0,tlc arthur's kindergarten learning system 2008,tlc arthur's kindergarten learning system is d...,encore,19.99,berlitz french premier language learning,system requirements: macintosh os x 10.2.8 or ...,,39.99,37.0,2011.0,6.0,0.195652,0.086739,0.000000
6,b000ecqxjq,http://www.google.com/base/feeds/snippets/1828...,0.137249,0.288220,0.123527,0.0,sims 2 nightlife expansion pack,sims 2: nightlife takes your sim into the nigh...,aspyr media,34.99,sims 2 pets for mac,system requirements: requires the full version...,,34.99,18.0,361.0,10.0,0.419355,0.238397,0.090909
7,b00004swln,http://www.google.com/base/feeds/snippets/1290...,0.091819,0.152497,0.122961,0.0,vpn manager five to ten firebox,watchguard vpn manager provides centralized ma...,watchguard technologies inc,1995.0,wg3605 watchguard mobile user vpn - complete p...,watchguard technologies wg3605 : usually ships...,,290.68,71.0,863.0,26.0,0.183908,0.153091,0.037037
8,b0006b6b1a,http://www.google.com/base/feeds/snippets/1837...,0.624055,0.634602,0.237562,1.0,business plan writer deluxe 8.0 2005,business plan writer deluxe 8 helps you get yo...,nova development,99.99,nova development bsw business plan writer delu...,business plan writer deluxe 8.0 bsw nova devel...,nova development,94.99,26.0,375.0,0.0,0.500000,0.115566,1.000000
9,b000ap41x2,http://www.google.com/base/feeds/snippets/1140...,0.053589,0.160767,0.000000,0.0,rayman 10th anniversary collection,three critically acclaimed action-packed games...,ubi soft,19.99,git captain america: the complete collection d...,,,42.99,35.0,82.0,7.0,0.326923,0.012048,0.125000


In [47]:
new_candidate_pairs_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31741 entries, 0 to 31740
Data columns (total 20 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   amazon_id                    31741 non-null  object 
 1   google_id                    31741 non-null  object 
 2   avg_similarity               31741 non-null  float64
 3   name_similarity              31741 non-null  float64
 4   description_similarity       31741 non-null  float64
 5   manufacturer_similarity      31741 non-null  float64
 6   amazon_name                  31741 non-null  object 
 7   amazon_description           31741 non-null  object 
 8   amazon_manufacturer          31741 non-null  object 
 9   amazon_price                 31741 non-null  object 
 10  google_name                  31741 non-null  object 
 11  google_description           31741 non-null  object 
 12  google_manufacturer          31741 non-null  object 
 13  google_price    


**2.2 [15 points]** Refer to the `Amazon_GoogleProducts_perfectMapping.csv` file for the actual matches. Create a new column named 'match' in your "candidate_pairs" DataFrame. If a pair of records (in the candidate set) corresponds to a match i.e., is in the perfectMapping file, label it as 'yes' in the 'match' column. Otherwise, label it as 'no.' Note that, by design, most pairs will actually be no! Furthermore, not every pair in the perfectMapping will be included here. What is the number of ‘false negatives’ that are not included in your dataset? 


In [48]:
merged_df = pd.merge(new_candidate_pairs_df, perfect_match, how='left', left_on=['amazon_id', 'google_id'], right_on=['idAmazon', 'idGoogleBase'])
new_candidate_pairs_df['match'] = np.where(merged_df['idAmazon'].notnull(), 1, 0)
false_negatives = len(perfect_match) - len(new_candidate_pairs_df[new_candidate_pairs_df['match']==1])

print("Number of FN: ", false_negatives)

Number of FN:  75


### Question 3: Feature Engineering for Machine Learning

**[10 points]** Prepare your feature matrix (X) and labels (y) for the machine learning model. 

Each row in X should correspond to a pair of records and contain the 

(i) three Levenshtein features calculated in Question 1; <br>
(ii) the three cosine similarity features you had computed earlier in HW5; and each entry in y should be the label indicating whether the pair of records is a match. Below is a feature set for each pair, including:

* name_lev: Levenshtein distance between product names.
* description_lev: Levenshtein distance between product descriptions.
* manufacturer_lev: Levenshtein distance between manufacturer names.
* name_tfidf_sim: Cosine similarity of TF-IDF vectors for product names.
* description_tfidf_sim: Cosine similarity of TF-IDF vectors for product descriptions.
* manufacturer_tfidf_sim: Cosine similarity of TF-IDF vectors for manufacturer names.

Print the first five vectors of matrix (X).

In [49]:
X = new_candidate_pairs_df[['name_similarity', 'description_similarity','manufacturer_similarity','name_lev_similarity', 'description_lev_similarity', 'manufacturer_lev_similarity']]
X.head(5)

,name_similarity,description_similarity,manufacturer_similarity,name_lev_similarity,description_lev_similarity,manufacturer_lev_similarity
0,0.245919,0.023959,0.0,0.236842,0.244353,0.000000
1,0.329629,0.172081,0.0,0.236486,0.220430,0.066667
2,0.194663,0.054457,0.0,0.232143,0.096360,0.052632
3,1.000000,0.525485,0.0,0.697674,0.381818,0.090909
4,0.146909,0.000000,0.0,0.328358,0.061093,0.066667


In [50]:
y=new_candidate_pairs_df['match']

### Question 4: Machine Learning for Entity Resolution

**4.1 [15 points]** : Randomly partition the matrix into a training set containing 80% of duplicates and non-duplicates, and a test set containing the rest. The training set must remain fixed for the experiments in 3.2. In running your code, verify that approximately 80% of the matches and non-matches in your original dataset compiled at the end of Q1 are in the training set. 

In [51]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, stratify=y)

In [52]:
len(X_train)/len(new_candidate_pairs_df)

0.7999747960051669

**4.2 [15 points]** : Select any three different machine learning models from the sklearn package (e.g., a decision tree-based model, a logistic regression model…) . Train each model using the feature set prepared in Question 2. You will end up with a total of three trained models. 

In [53]:
dt_model = DecisionTreeClassifier(random_state=2)
dt_model.fit(X_train, y_train)
dt_predictions = dt_model.predict(X_test)

rf_model = RandomForestClassifier(random_state=2)
rf_model.fit(X_train, y_train)
rf_predictions = rf_model.predict(X_test)

gb_model = GradientBoostingClassifier(random_state=2)
gb_model.fit(X_train, y_train)
gb_predictions = gb_model.predict(X_test)

**4.3 [10 points]** : Provide a table below where, for each method, you are providing the AUC, precision, recall, F-measure and accuracy. Based on the numbers, which of these metrics might be problematic for evaluating ER? Why?



In [54]:
dt_precision = precision_score(y_test, dt_predictions)
dt_recall = recall_score(y_test, dt_predictions)
dt_f1 = f1_score(y_test, dt_predictions)
dt_auc = roc_auc_score(y_test, dt_predictions)
dt_accuracy = accuracy_score(y_test, dt_predictions)

rf_precision = precision_score(y_test, rf_predictions)
rf_recall = recall_score(y_test, rf_predictions)
rf_f1 = f1_score(y_test, rf_predictions)
rf_auc = roc_auc_score(y_test, rf_predictions)
rf_accuracy = accuracy_score(y_test, rf_predictions)

gb_precision = precision_score(y_test, gb_predictions)
gb_recall = recall_score(y_test, gb_predictions)
gb_f1 = f1_score(y_test, gb_predictions)
gb_auc = roc_auc_score(y_test, gb_predictions)
gb_accuracy = accuracy_score(y_test, gb_predictions)

In [55]:
evaluation_data = {
    'Method': ['Decision Tree', 'Random Forest', 'Gradient Boost'],
    'AUC': [dt_auc, rf_auc, gb_auc],
    'Precision': [dt_precision, rf_precision, gb_precision],
    'Recall': [dt_recall, rf_recall, gb_recall],
    'F-measure': [dt_f1, rf_f1, gb_f1],
    'Accuracy': [dt_accuracy, rf_accuracy, gb_accuracy]
}

evaluation_df = pd.DataFrame(evaluation_data)

print(evaluation_df)

           Method       AUC  Precision    Recall  F-measure  Accuracy
0   Decision Tree  0.681836   0.392562  0.387755   0.390144  0.953221
1   Random Forest  0.675742   0.651852  0.359184   0.463158  0.967869
2  Gradient Boost  0.623336   0.613861  0.253061   0.358382  0.965034


I think just using precision or recall will cause problems. Because the dataset is very imbalance, just using one of them can not provide enough evaluating information.

**4.4 [10 points]** : Using paired t-tests, can you indicate which (if any) of the three models is significantly better than each of the other two? 

In [67]:
t_statistic, p_value_1_2 = ttest_rel(evaluation_df.iloc[0,:].to_list()[1:], evaluation_df.iloc[1,:].to_list()[1:], alternative='greater')
t_statistic, p_value_1_3 = ttest_rel(evaluation_df.iloc[0,:].to_list()[1:], evaluation_df.iloc[2,:].to_list()[1:], alternative='greater')
t_statistic, p_value_2_3 = ttest_rel(evaluation_df.iloc[1,:].to_list()[1:], evaluation_df.iloc[2,:].to_list()[1:], alternative='greater')

print("p-value (1_2):", round(p_value_1_2,5))
print("p-value (1_3):", round(p_value_1_3,5))
print("p-value (2_3):", round(p_value_2_3,5))

p-value (1_2): 0.85193
p-value (1_3): 0.51022
p-value (2_3): 0.01896


 Based on the evaluation metrics, Random Forest model is significantly better than others.